# Multi-Agents Plan Execute – README 😊

## Обзор 🚀

Данный проект демонстрирует комплексную систему, которая объединяет планирование, выполнение и перепланирование шагов для достижения окончательного ответа по заданной цели. Система использует инструменты поиска, языковые модели AzureChatOpenAI и специализированные компоненты для создания пошагового плана, его выполнения и обновления. Такой подход позволяет автоматизировать процесс получения качественных ответов от агентов. 👍

## Цель проекта 🎯

Основная задача проекта – обеспечить:
- **Планирование.** Автоматическое формирование простого и логически выверенного пошагового плана для достижения поставленной цели.
- **Выполнение.** Пошаговое выполнение плана с помощью агента-исполнителя, который может использовать внешние инструменты поиска для получения дополнительной информации.
- **Перепланирование.** Обновление и корректировку плана с учётом уже выполненных шагов до получения окончательного результата.
- **Финальный ответ.** Формирование окончательного ответа для пользователя после выполнения всех необходимых шагов.

## Основные компоненты 🛠

### 1. Настройка окружения 🌐
- **Переменные окружения.** Используется `dotenv` для загрузки настроек, необходимых для работы с API Azure и другими компонентами.
- **Инструменты поиска.** Интеграция с инструментами, такими как `TavilySearchResults` (а также можно использовать DuckDuckGo), для получения актуальной информации.

### 2. Агенты и планировщики 🤖
- **Агент-исполнитель.** Создаётся с помощью функции `create_react_agent` и выполняет задания, используя переданный промпт и инструменты поиска.
- **Планировщик.** Составляет простой пошаговый план для достижения цели, используя промпт, заданный через `ChatPromptTemplate`.
- **Перепланировщик.** Корректирует план с учётом уже выполненных шагов и обновляет его, если необходимо.

### 3. Рабочий процесс с графом состояний 🔄
- **Граф состояний (StateGraph).** Определяет последовательность шагов:
  - **plan_step.** Генерация плана на основе входного запроса.
  - **execute_step.** Выполнение первого шага плана с использованием агента.
  - **replan_step.** Перепланирование с учётом уже выполненных шагов.
- **Условие завершения.** Процесс продолжается до тех пор, пока не сформируется окончательный ответ, который возвращается пользователю.

### 4. Асинхронное выполнение 📡
- **Асинхронные функции.** Используются для планирования, выполнения и перепланирования шагов, что обеспечивает эффективное и параллельное выполнение задач.
- **Поток событий.** Рабочий процесс запускается в асинхронном режиме, позволяя последовательно обрабатывать каждый этап и выводить промежуточные результаты.

## Как запустить проект 🚀

1. **Настройка окружения.** Создайте файл `.env` с необходимыми переменными (например, `AZURE_OPENAI_ENDPOINT`, `AZURE_OPENAI_API_VERSION` и т.д.).
2. **Установка зависимостей.** Убедитесь, что установлены все необходимые библиотеки: `dotenv`, `langchain`, `langchain_community`, `langgraph`, `pydantic` и прочие.
3. **Запуск в Jupyter Notebook.** Откройте тетрадку в Jupyter Notebook или другом асинхронном окружении, выполните все ячейки последовательно. Граф рабочего процесса отобразится с помощью mermaid, а финальный результат будет выведен в консоль. 💻

## Заключение ✨

Этот проект демонстрирует, как с помощью многоагентной архитектуры можно автоматизировать сложные процессы планирования, выполнения и перепланирования для достижения окончательного ответа по заданной цели. Такой подход позволяет повысить качество ответов и упростить интеграцию различных инструментов поиска и обработки данных. Надеюсь, этот пример будет полезен для ваших исследований и разработок! 😊

## Load Env Vars

In [ ]:
# Импорт библиотек

# Импорт библиотек для работы с переменными окружения
from dotenv import load_dotenv
import os

# Импорт библиотек для работы с инструментами поиска
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.tools import DuckDuckGoSearchResults

# Импорт библиотек для работы с моделями и агентами
from langchain import hub
from langchain_openai import AzureChatOpenAI
from langgraph.prebuilt import create_react_agent

# Импорт библиотек для работы с типизацией и аннотациями
import operator
from typing import Annotated, List, Tuple, TypedDict, Union, Literal

# Импорт библиотек для работы с моделями данных
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate

# Импорт библиотек для работы с графами состояний
from langgraph.graph import StateGraph, START

# Импорт библиотек для отображения изображений
from IPython.display import Image, display

# Загрузка переменных окружения из файла .env
load_dotenv()
os.environ["LANGCHAIN_PROJECT"] = "Multi-Agents-Plan-Execute"

## Web Searching Tool


In [ ]:
# Настройка инструментов поиска
tools = [TavilySearchResults(max_results=10)]

## Create Agent

In [ ]:
# Настройка модели Azure Chat OpenAI
deployment_name = 'gpt-4o-mini'
llm = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=deployment_name,
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
)

# Создание агента-исполнителя
agent_executor = create_react_agent(llm, tools, messages_modifier=prompt)

Test a simple call

In [ ]:
# Вызов агента-исполнителя
agent_executor.invoke(
    {
        "messages": [("user", "best places to visit in beijing")]
    }
)

## Define PlanExecute State

In [ ]:
# Определение типа данных для плана выполнения
class PlanExecute(TypedDict):
    input: str
    plan: List[str]
    past_steps: Annotated[List[Tuple], operator.add]
    response: str

## Planning Step

In [ ]:
# Определение модели данных для плана
class Plan(BaseModel):
    """План для выполнения в будущем"""

    steps: List[str] = Field(
        description="Различные шаги для выполнения, должны быть в отсортированном порядке"
    )

In [ ]:
# Настройка промпта для планировщика
planner_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
                Для заданной цели придумайте простой пошаговый план.
                Этот план должен включать индивидуальные задачи, которые при правильном выполнении приведут к правильному ответу.
                Не добавляйте лишних шагов.
                Результат последнего шага должен быть окончательным ответом.
                Убедитесь, что каждый шаг содержит всю необходимую информацию - не пропускайте шаги.
            """,
        ),
        ("placeholder", "{messages}"),
    ]
)

# Создание планировщика
planner = planner_prompt | AzureChatOpenAI(
    temperature=0,
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=deployment_name,
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
).with_structured_output(Plan)

In [ ]:
# Вызов планировщика
plan = planner.invoke(
    {
        "messages": [
            ("user", "Согласно самым известным туристическим справочникам, какие 10 мест в Пекине, Китай, стоит посетить?")
        ]
    }
)

# Вывод шагов плана
for step in plan.steps:
    print(step)

## Re-Plan Step


In [ ]:
# Определение модели данных для ответа
class Response(BaseModel):
    """Ответ пользователю."""

    response: str

# Определение модели данных для действия
class Act(BaseModel):
    """Действие для выполнения."""

    action: Union[Response, Plan] = Field(
        description="Действие для выполнения. Если хотите ответить пользователю, используйте Response. "
                    "Если нужно использовать инструменты для получения ответа, используйте Plan."
    )

# Настройка промпта для перепланировщика
replanner_prompt = ChatPromptTemplate.from_template("""
    Для заданной цели придумайте простой пошаговый план.
    Этот план должен включать индивидуальные задачи, которые при правильном выполнении приведут к правильному ответу.
    Не добавляйте лишних шагов, держите все просто.
    Результат последнего шага должен быть окончательным ответом.
    Убедитесь, что каждый шаг содержит всю необходимую информацию - не пропускайте шаги.

    Ваша цель была следующей:
    {input}

    Ваш первоначальный план был следующим:
    {plan}

    Вы уже выполнили следующие шаги:
    {past_steps}

    Обновите ваш план соответственно. Если больше шагов не требуется и вы можете вернуться к пользователю, то ответьте ему.
    В противном случае заполните план. Предпочтительнее вернуться к пользователю, вместо того чтобы создавать ненужные шаги.
    Не возвращайте ранее выполненные шаги в качестве части плана.
    """
)

# Создание перепланировщика
replanner = replanner_prompt | AzureChatOpenAI(
    temperature=0,
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=deployment_name,
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
).with_structured_output(Act)

## Plan the Graph


In [ ]:
# Асинхронная функция для выполнения шага
async def execute_step(state: PlanExecute) -> dict:
    """
    Description:
        Выполняет шаг плана.

    Args:
        state (PlanExecute): Текущее состояние выполнения плана.

    Returns:
        dict: Обновленное состояние с выполненным шагом.

    Examples:
        >>> state = {"plan": ["step1", "step2"], "past_steps": []}
        >>> await execute_step(state)
        {'past_steps': [('step1', 'response_content')]}
    """
    print('state : ', state)

    plan = state["plan"]
    plan_str = "\n".join(f"{i+1}. {step}" for i, step in enumerate(plan))
    task = plan[0]
    task_formatted = (
        f"Для следующего плана:\n{plan_str}\n\n"
        f"Вам поручено выполнить шаг {1}, {task}."
    )

    agent_response = await agent_executor.ainvoke(
        {"messages": [("user", task_formatted)]}
    )
    return {
        "past_steps": [(task, agent_response["messages"][-1].content)],
    }

# Асинхронная функция для планирования шага
async def plan_step(state: PlanExecute) -> dict:
    """
    Description:
        Планирует шаги для выполнения.

    Args:
        state (PlanExecute): Текущее состояние выполнения плана.

    Returns:
        dict: Обновленное состояние с планом шагов.

    Examples:
        >>> state = {"input": "Какие 5 лучших мест стоит посетить в Пекине, Китай?"}
        >>> await plan_step(state)
        {'plan': ['step1', 'step2', 'step3']}
    """
    plan = await planner.ainvoke({"messages": [("user", state["input"])]})
    return {"plan": plan.steps}

# Асинхронная функция для перепланирования шага
async def replan_step(state: PlanExecute) -> dict:
    """
    Description:
        Перепланирует шаги для выполнения.

    Args:
        state (PlanExecute): Текущее состояние выполнения плана.

    Returns:
        dict: Обновленное состояние с планом шагов или ответом.

    Examples:
        >>> state = {"input": "Какие 5 лучших мест стоит посетить в Пекине, Китай?"}
        >>> await replan_step(state)
        {'plan': ['step1', 'step2', 'step3']}
    """
    output = await replanner.ainvoke(state)
    if isinstance(output.action, Response):
        return {"response": output.action.response}
    else:
        return {"plan": output.action.steps}

# Функция для проверки завершения
def should_end(state: PlanExecute) -> Literal["agent", "__end__"]:
    """
    Description:
        Проверяет, следует ли завершить выполнение плана.

    Args:
        state (PlanExecute): Текущее состояние выполнения плана.

    Returns:
        Literal["agent", "__end__"]: Возвращает "agent", если выполнение должно продолжаться,
                                     или "__end__", если выполнение должно быть завершено.

    Examples:
        >>> state = {"response": "Ответ пользователю"}
        >>> should_end(state)
        '__end__'
    """
    if "response" in state and state["response"]:
        return "__end__"
    else:
        return "agent"

In [ ]:
# Создание рабочего процесса
workflow = StateGraph(PlanExecute)

# Добавление узла планировщика
workflow.add_node("planner", plan_step)

# Добавление узла выполнения
workflow.add_node("agent", execute_step)

# Добавление узла перепланировщика
workflow.add_node("replan", replan_step)

# Добавление ребер графа
workflow.add_edge(START, "planner")
workflow.add_edge("planner", "agent")
workflow.add_edge("agent", "replan")

# Добавление условных ребер для завершения
workflow.add_conditional_edges(
    "replan",
    should_end,
)

# Компиляция рабочего процесса
app = workflow.compile()

In [ ]:
# Отображение графа рабочего процесса
display(Image(app.get_graph(xray=True).draw_mermaid_png()))


In [ ]:
# Настройка конфигурации
config = {"recursion_limit": 100}

# Входные данные для рабочего процесса
inputs = {"input": "Какие 5 лучших мест стоит посетить в Пекине, Китай? "}

# Асинхронный поток событий рабочего процесса
async for event in app.astream(inputs, config=config):
    for k, v in event.items():
        if k != "__end__":
            print(v)

In [ ]:
print(v['response'])